<a href="https://colab.research.google.com/github/EnFiore/Master-AI/blob/main/gestione_magazzino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import json
import os

In [1]:
#Recupero i dati dal magazzino
def get_products_list(file_json):
    try:
        with open(file_json, 'r') as f:
            data_wr = json.load(f)
    except FileNotFoundError:
        return 'il magazzino è vuoto'
    elenco=""
    print("PRODOTTO, QUANTITA', PREZZO")
    for d in data_wr:
        print(d['name'] +' ' + str(d['quantity']) +' €'+ str(d['sell_price']))

#Recupero il prezzo di vendita
def get_sell_price(file_json,values):
    with open(file_json, 'r') as f:
        data_wr = json.load(f)
    for d in data_wr:
        if d['name'] == values[0]:
            return d['sell_price']

#Aggiungo prodotti dal magazzino
def add_products(file_json, values):
    if os.path.exists(magazzino):
        with open(magazzino, "r+") as f:
            wr_dict = json.load(f)
    else:
        wr_dict = []
    if len(values) > 2:
        key_value = ["name", "quantity","buy_price","sell_price","qt_sell"]
        values.append(0)
        product_dict = dict(zip(key_value, values))
        wr_dict.append(product_dict)
        with open(magazzino, "w") as f:
            json.dump(wr_dict, f,indent=4)
        print(f"Aggiunto: {str(product_dict['quantity'])} x {str(product_dict['name'])}")
    else:
        for d in wr_dict:
            if d['name'] == values[0]:
                d['quantity'] += values[1]
                break
        with open(magazzino, "w") as f:
            json.dump(wr_dict, f,indent=4)
        print(f"Aggiunto: {str(values[0])} x {str(values[1])}")

# Vendo il prodotto
def sell_products(file_json, values):
    if os.path.exists(file_json):
        with open(file_json, "r+") as f:
            wr_dict = json.load(f)
    else:
        return 'no_mag'
    for d in wr_dict:
        if d['name'] == values[0]:
            if d['quantity'] >= values[1]:
                d['quantity'] -= values[1]
                d['qt_sell'] += values[1]
                break
            else:
                return "no_prod"
    with open(file_json, "w") as f:
            json.dump(wr_dict, f,indent=4)
    return 'ok'

#calcolo il prezzo lordo
def get_gross_profit(file_json):
    if os.path.exists(file_json):
        with open(file_json, "r+") as f:
            wr_dict = json.load(f)
    else:
        return 'no_mag'
    total=0
    for d in wr_dict:
        total += d['qt_sell']* d["sell_price"]
    return round(total,2)

# calcolo il costo totale
def get_total_cost(file_json):
    if os.path.exists(file_json):
        with open(file_json, "r+") as f:
            wr_dict = json.load(f)
    else:
        return 'no_mag'
    total=0
    for d in wr_dict:
        total += (d['qt_sell']+d['quantity'])* d["buy_price"]
    return round(total,2)

# controllo che il prodotto sia nel magazzino
def check_product(file_json,prodotto):
    try:
        risposta = False
        with open(file_json, 'r+') as f:
            data_wr = json.load(f)
            for d in data_wr:
                if d['name'] == prodotto:
                    risposta = True
                    break
                else:
                    risposta = False
        return risposta
    except FileNotFoundError:
        return False
# Menù dei comandi
def show_command_menu():
    menu = '''Seleziona l'operazione che vuoi effettuare:
 - aggiungi: premette di aggiungere articoli al magazzino
 - elenca: permette di leggere il contenuto del magazzino
 - vendita: vendere un prodotto
 - profitti: calcola il profitto lordo e netto
 - aiuto: mostra questo menù
 - chiudi: uscire dal programma'''
    print(menu)

#Controllo che il comando inserito sia corretto
def check_command(operation):
    menu_command = ('aggiungi', 'elenca', 'vendita','profitti', 'aiuto', 'chiudi')
    if operation.lower() in menu_command:
        return True
    else:
        print('''***\nATTENZIONE COMANDO INSERITO ERRATO!\nINSERIRE IL COMANDO CORRETO.\n***\n''')
        show_command_menu()
        return False

def is_float(value):
    if value.replace(".", "").isnumeric():
        return True
    else:
        return False



In [4]:
magazzino = "magazzino.json"
no_insert = "Inserimento prodotti terminato.\nil prodotto non è stato inserito"
show_command_menu()
exit = False
while exit == False:
    operation = input("Digita l'operazione che vuoi effettuare: ")
    if check_command(operation):
        #Aggiungo prodotto
        if operation.lower() == "aggiungi":
            continues = 'si'
            while continues == 'si':
                values=[]
                name = input('Inserisci il nome del prodotto: ')
                values.append(name)
                quantity= input('''Inserire la quantità: ''')
                while not quantity.isdigit():
                    if not quantity == "exit":
                        print("Il valore della quantità deve essere un numero intero\n")
                        quantity = input("Inserire il valore corretto o exit per uscire:")
                    else:
                        print(no_insert)
                        break
                if not quantity == 'exit':
                    if check_product(magazzino,name):
                        values.append(int(quantity))
                        print('prodotto presente in magazzino. Sarà aggiornata la quantità')
                        add_products(magazzino, values)
                        r = input('Aggiungere un altro prodotto?(si/no)')
                        if r.lower()=='si' or r.lower()=='no':
                                  continues = r
                        else:
                            print('Risposta non valida')
                            continues='no'
                    else:
                        values.append(int(quantity))
                        buy_price = input("inserire prezzo acquisto: ")
                        if is_float(buy_price):
                            values.append(float(buy_price))
                        else:
                            while not is_float(buy_price):
                                if not buy_price == 'exit':
                                    print("Il prezzo d'acquisto deve essere un numero\n")
                                    buy_price = input("Inserire il valore corretto o exit per uscire:")
                                elif buy_price == 'exit':
                                    print(no_insert)
                                    break
                            values.append(float(buy_price))
                        if not buy_price == 'exit':
                            sell_price = input("inserire prezzo vendita: ")
                            if is_float(sell_price):
                                values.append(float(sell_price))
                                add_products(magazzino, values)
                                r = input('Aggiungere un altro prodotto?(si/no)')
                                if r.lower()=='si' or r.lower()=='no':
                                          continues = r
                                else:
                                    print('Risposta non valida')
                                    continues='no'
                            else:
                                while not is_float(sell_price):
                                    if not sell_price == 'exit':
                                        print("Il prezzo di vendita deve essere un numero\n")
                                        sell_price= input("Inserire il valore corretto o exit per uscire: ")
                                    elif sell_price == 'exit':
                                        print(no_insert)
                                        break
                                values.append(float(sell_price))
                                if not sell_price == 'exit':
                                    add_products(magazzino, values)
                                    r = input('Aggiungere un altro prodotto?(si/no)')
                                    if r.lower()=='si' or r.lower()=='no':
                                              continues = r
                                    else:
                                        print('Risposta non valida')
                                        continues='no'
         # Elenco prodotti
        elif operation.lower() == "elenca":
            print(get_products_list(magazzino))

        # Vendita prodotto
        elif operation.lower() == "vendita":
            values=[]
            name = input('Inserisci il nome del prodotto')
            values.append(name)
            quantity= input('''Inserire la quantità:''')
            while not quantity.isdigit():
                if not quantity == "exit":
                    print("Il valore della quantità deve essere un numero intero\n")
                    quantity = input("Inserire il valore corretto o exit per uscire: ")
                else:
                    print(no_insert)
                    break
            if not quantity == 'exit':
                if check_product(magazzino,name):
                    values.append(int(quantity))
                    sell = sell_products(magazzino, values)
                    if sell == 'ok':
                        print('VENDITA REGISTRATA')
                        sell_price = get_sell_price(magazzino,values)
                        print(f"{values[1]} X {values[0]}: {sell_price}")
                        totale = round(values[1]*sell_price,2)
                        print(f"Totale: €{totale}")
                    elif sell == 'no_mag':
                        print('Magazzino vuoto')
                    elif sell == 'no_prod':
                        print('Non hai abbastabza prodotti da vendere')
                else:
                    print('prodotto non trovato. vendita non effettuata')

        # Calcolo profitti
        elif operation.lower() == "profitti":

            print(f"Profitto: Lordo=€{get_gross_profit(magazzino)} netto=€{round(get_gross_profit(magazzino)-get_total_cost(magazzino),2)}")

        # Mostra menù
        elif operation.lower() == "aiuto":
            show_command_menu()

        # Esce dal programma
        elif operation.lower() == "chiudi":
            print("Bye, bye")
            exit = True
quit()

Seleziona l'operazione che vuoi effettuare:
 - aggiungi: premette di aggiungere articoli al magazzino
 - elenca: permette di leggere il contenuto del magazzino
 - vendita: vendere un prodotto
 - profitti: calcola il profitto lordo e netto
 - aiuto: mostra questo menù
 - chiudi: uscire dal programma
Digita l'operazione che vuoi effettuare: aggiungi
Inserisci il nome del prodotto: formaggio
Inserire la quantità: 32
inserire prezzo acquisto: 5
inserire prezzo vendita: 15
Aggiunto: 32 x formaggio
Aggiungere un altro prodotto?(si/no)no
Digita l'operazione che vuoi effettuare: elenca
PRODOTTO, QUANTITA', PREZZO
formaggio 32 €15.0
None
Digita l'operazione che vuoi effettuare: chiudi
Bye, bye
